In [34]:
import daft
df_all = daft.read_parquet("s3://daft-public-data/cifar100/cifar_images.parquet")
df_all = df.with_column(
    "image", 
    (df["image_url"]
     .url.download()
     .image.decode()
     .image.resize(224, 224) # Resize for the model
    )
)
df_all.show()

╭───────┬────────────────────────────────┬───────────────┬──────────────╮
│ id    ┆ image_url                      ┆ category      ┆ image        │
│ ---   ┆ ---                            ┆ ---           ┆ ---          │
│ Int64 ┆ Utf8                           ┆ Utf8          ┆ Image[MIXED] │
╞═══════╪════════════════════════════════╪═══════════════╪══════════════╡
│ 0     ┆ s3://daft-public-data/cifar10… ┆ cattle        ┆ <Image>      │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1     ┆ s3://daft-public-data/cifar10… ┆ dinosaur      ┆ <Image>      │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2     ┆ s3://daft-public-data/cifar10… ┆ apple         ┆ <Image>      │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3     ┆ s3://daft-public-data/cifar10… ┆ boy           ┆ <Image>      │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4     ┆ s3://daft-public-data/cifar10… ┆ aquarium_fish ┆ <Image>      │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5     ┆ s3://daft-public-data/cifar10… ┆ telephone     ┆ <Image>      │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 6     ┆ s3://daft-public-data/cifar10… ┆ train         ┆ <Image>      │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 7     ┆ s3://daft-public-data/cifar10… ┆ cup           ┆ <Image>      │
╰───────┴────────────────────────────────┴───────────────┴──────────────╯

(Showing first 8 rows)

In [35]:
import torch
from torch.utils.data import DataLoader
import torchvision.models as models

import timm

# Load ViT base 32 from timm — same backbone as CLIP
model = timm.create_model("vit_base_patch32_224", pretrained=True)
model.reset_classifier(0)  # remove final head
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval().to(device)

torch_dataset = df_all.to_torch_iter_dataset()
dataloader = DataLoader(torch_dataset, batch_size=16)

embeddings = []
ids = []

for batch in iter(dataloader):
    image_tensor = batch["image"].to(device)
    image_tensor = image_tensor.permute(0, 3, 1, 2).float() / 255.0
    outputs = model(image_tensor)
    outputs = outputs.view(outputs.size(0), -1)
    embeddings.append(outputs.cpu())
    ids += batch["id"].tolist()

all_embeddings = torch.cat(embeddings, dim=0)
embedding_list = all_embeddings.detach().numpy().tolist()
embedding_df = daft.from_pydict({
    "id": ids,
    "embedding": embedding_list,
})
embedding_df.show()

╭───────┬────────────────────────────────╮
│ id    ┆ embedding                      │
│ ---   ┆ ---                            │
│ Int64 ┆ List[Float64]                  │
╞═══════╪════════════════════════════════╡
│ 0     ┆ [0.82135009765625, 0.58786773… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1     ┆ [-1.1168181896209717, -1.2613… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2     ┆ [-2.262598991394043, 0.293584… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3     ┆ [-1.2193759679794312, 1.62847… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4     ┆ [2.0632448196411133, 0.411833… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5     ┆ [-1.6371591091156006, -3.0188… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 6     ┆ [-0.3102252781391144, -0.5160… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 7     ┆ [-0.5268806219100952, 0.07281… │
╰───────┴────────────────────────────────╯

(Showing first 8 of 1000 rows)

In [36]:
df_all = df_all.join(embedding_df, on="id")
df_all.collect()

🗡️ 🐟 Project: 00:00 

🗡️ 🐟 HashJoinBuild: 00:00 

🗡️ 🐟 InnerHashJoinProbe: 00:00 

╭───────┬────────────────────────────────┬───────────────┬──────────────┬────────────────────────────────╮
│ id    ┆ image_url                      ┆ category      ┆ image        ┆ embedding                      │
│ ---   ┆ ---                            ┆ ---           ┆ ---          ┆ ---                            │
│ Int64 ┆ Utf8                           ┆ Utf8          ┆ Image[MIXED] ┆ List[Float64]                  │
╞═══════╪════════════════════════════════╪═══════════════╪══════════════╪════════════════════════════════╡
│ 0     ┆ s3://daft-public-data/cifar10… ┆ cattle        ┆ <Image>      ┆ [0.82135009765625, 0.58786773… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1     ┆ s3://daft-public-data/cifar10… ┆ dinosaur      ┆ <Image>      ┆ [-1.1168181896209717, -1.2613… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2     ┆ s3://daft-public-data/cifar10… ┆ apple         ┆ <Image>      ┆ [-2.262598991394043, 0.293584… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3     ┆ s3://daft-public-data/cifar10… ┆ boy           ┆ <Image>      ┆ [-1.2193759679794312, 1.62847… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4     ┆ s3://daft-public-data/cifar10… ┆ aquarium_fish ┆ <Image>      ┆ [2.0632448196411133, 0.411833… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5     ┆ s3://daft-public-data/cifar10… ┆ telephone     ┆ <Image>      ┆ [-1.6371591091156006, -3.0188… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 6     ┆ s3://daft-public-data/cifar10… ┆ train         ┆ <Image>      ┆ [-0.3102252781391144, -0.5160… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 7     ┆ s3://daft-public-data/cifar10… ┆ cup           ┆ <Image>      ┆ [-0.5268806219100952, 0.07281… │
╰───────┴────────────────────────────────┴───────────────┴──────────────┴────────────────────────────────╯

(Showing first 8 of 1000 rows)

In [37]:
single_embedding = next(iter(df_all))["embedding"]

In [38]:
len(single_embedding)

768

In [39]:
dtype = daft.DataType.fixed_size_list(dtype=daft.DataType.float64(), size=768)

In [40]:
df_all = (df_all
          .select("id", "category", "image", "embedding")
          .with_column_renamed("category", "original_category")
          .with_column("embedding", daft.col("embedding").cast(dtype))
         )
df_all.show()

╭───────┬───────────────────┬──────────────┬────────────────────────────────╮
│ id    ┆ original_category ┆ image        ┆ embedding                      │
│ ---   ┆ ---               ┆ ---          ┆ ---                            │
│ Int64 ┆ Utf8              ┆ Image[MIXED] ┆ FixedSizeList[Float64; 768]    │
╞═══════╪═══════════════════╪══════════════╪════════════════════════════════╡
│ 0     ┆ cattle            ┆ <Image>      ┆ [0.82135009765625, 0.58786773… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 1     ┆ dinosaur          ┆ <Image>      ┆ [-1.1168181896209717, -1.2613… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 2     ┆ apple             ┆ <Image>      ┆ [-2.262598991394043, 0.293584… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 3     ┆ boy               ┆ <Image>      ┆ [-1.2193759679794312, 1.62847… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 4     ┆ aquarium_fish     ┆ <Image>      ┆ [2.0632448196411133, 0.411833… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 5     ┆ telephone         ┆ <Image>      ┆ [-1.6371591091156006, -3.0188… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 6     ┆ train             ┆ <Image>      ┆ [-0.3102252781391144, -0.5160… │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 7     ┆ cup               ┆ <Image>      ┆ [-0.5268806219100952, 0.07281… │
╰───────┴───────────────────┴──────────────┴────────────────────────────────╯

(Showing first 8 rows)

In [41]:
canonical_df = daft.read_parquet("s3://daft-public-data/cifar100/canonical_images.parquet/")
canonical_df = (canonical_df
                .with_column("embedding", daft.col("embedding").cast(dtype))
                .with_columns_renamed({
                    "category": "canonical_category",
                    "image": "canonical_image",
                    "embedding": "canonical_embedding"
                })
               )
canonical_df.show()

╭────────────────────┬─────────────────┬────────────────────────────────╮
│ canonical_category ┆ canonical_image ┆ canonical_embedding            │
│ ---                ┆ ---             ┆ ---                            │
│ Utf8               ┆ Image[MIXED]    ┆ FixedSizeList[Float64; 768]    │
╞════════════════════╪═════════════════╪════════════════════════════════╡
│ cattle             ┆ <Image>         ┆ [0.82135009765625, 0.58786773… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ lizard             ┆ <Image>         ┆ [-0.5083964467048645, 0.23103… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ beaver             ┆ <Image>         ┆ [-1.7283765077590942, -0.2562… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ whale              ┆ <Image>         ┆ [0.08972498029470444, -0.6458… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ elephant           ┆ <Image>         ┆ [-2.5771164894104004, 0.57594… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ snake              ┆ <Image>         ┆ [-0.827828586101532, -0.99769… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ squirrel           ┆ <Image>         ┆ [0.8686367273330688, 0.617227… │
├╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ shark              ┆ <Image>         ┆ [2.8686251640319824, -1.62483… │
╰────────────────────┴─────────────────┴────────────────────────────────╯

(Showing first 8 rows)

In [29]:
# Note: Window function support is coming to Daft soon which will make the following query much easier to express and more efficient to compute!
df_joined = df_all.join(canonical_df, how="cross")
df_joined = df_joined.with_column(
    "cosine_distance",
    daft.col("embedding").embedding.cosine_distance(daft.col("canonical_embedding"))
)
df_min_dist = df_joined.groupby("id").agg(
    daft.col("cosine_distance").min().alias("min_distance")
)
df_closest = df_joined.join(df_min_dist, left_on=["id", "cosine_distance"], right_on=["id", "min_distance"])
df_closest.collect()
df_closest = df_closest.where(daft.col("cosine_distance") < 0.35)
df_closest.show()

╭───────┬───────────────────┬──────────────┬────────────┬────────────────────┬────────────────────┬────────────────────╮
│ id    ┆ original_category ┆ image        ┆      …     ┆ canonical_embeddin ┆ cosine_distance    ┆ min_distance       │
│ ---   ┆ ---               ┆ ---          ┆            ┆ g                  ┆ ---                ┆ ---                │
│ Int64 ┆ Utf8              ┆ Image[MIXED] ┆ (3 hidden) ┆ ---                ┆ Float64            ┆ Float64            │
│       ┆                   ┆              ┆            ┆ FixedSizeList[Floa ┆                    ┆                    │
│       ┆                   ┆              ┆            ┆ t64; 768]          ┆                    ┆                    │
╞═══════╪═══════════════════╪══════════════╪════════════╪════════════════════╪════════════════════╪════════════════════╡
│ 0     ┆ cattle            ┆ <Image>      ┆ …          ┆ [0.82135009765625, ┆ 0                  ┆ 0                  │
│       ┆                   ┆              ┆            ┆ 0.58786773…        ┆                    ┆                    │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 9     ┆ elephant          ┆ <Image>      ┆ …          ┆ [-2.57711648941040 ┆ 0                  ┆ 0                  │
│       ┆                   ┆              ┆            ┆ 04, 0.57594…       ┆                    ┆                    │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 18    ┆ squirrel          ┆ <Image>      ┆ …          ┆ [0.868636727333068 ┆ 0                  ┆ 0                  │
│       ┆                   ┆              ┆            ┆ 8, 0.617227…       ┆                    ┆                    │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 34    ┆ chimpanzee        ┆ <Image>      ┆ …          ┆ [0.351638883352279 ┆ -0.000000000000000 ┆ -0.000000000000000 │
│       ┆                   ┆              ┆            ┆ 66, 1.28936…       ┆ 22204460492…       ┆ 22204460492…       │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 40    ┆ snake             ┆ <Image>      ┆ …          ┆ [-0.82782858610153 ┆ 0                  ┆ 0                  │
│       ┆                   ┆              ┆            ┆ 2, -0.99769…       ┆                    ┆                    │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 50    ┆ shark             ┆ <Image>      ┆ …          ┆ [2.868625164031982 ┆ 0                  ┆ 0                  │
│       ┆                   ┆              ┆            ┆ 4, -1.62483…       ┆                    ┆                    │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 72    ┆ camel             ┆ <Image>      ┆ …          ┆ [1.115436911582946 ┆ 0.0000000000000001 ┆ 0.0000000000000001 │
│       ┆                   ┆              ┆            ┆ 8, -1.45283…       ┆ 11022302462…       ┆ 11022302462…       │
├╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┼╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌╌┤
│ 94    ┆ whale             ┆ <Image>      ┆ …          ┆ [0.089724980294704 ┆ 0                  ┆ 0                  │
│       ┆                   ┆              ┆            ┆ 44, -0.6458…       ┆                    ┆                    │
╰───────┴───────────────────┴──────────────┴────────────┴────────────────────┴────────────────────┴────────────────────╯

(Showing first 8 rows)

In [25]:
df_closest.count_rows()

29

In [26]:
(df_closest
 .select("id", "original_category", "image", "canonical_category")
 .with_column_renamed("canonical_category", "classified_category")
 .write_parquet("bucketed_images.parquet", partition_cols=["classified_category"], write_mode="overwrite")
)

pathUtf8,classified_categoryUtf8
bucketed_images.parquet/classified_category=snake/b6c46dfc-f795-47d4-93ac-6ecbdba4c2aa-0.parquet,snake
bucketed_images.parquet/classified_category=shark/ff40b315-81f2-4097-9534-7a05a9d16f79-0.parquet,shark
bucketed_images.parquet/classified_category=camel/19c4072b-178e-423a-831d-0c2b0b84778f-0.parquet,camel
bucketed_images.parquet/classified_category=chimpanzee/5ccbaed7-74c2-4ca4-b809-7aabb012f4ed-0.parquet,chimpanzee
bucketed_images.parquet/classified_category=squirrel/e25ffc40-806f-4eed-9c84-a9a581719e21-0.parquet,squirrel
bucketed_images.parquet/classified_category=elephant/934b3470-01f6-4c11-ad85-9fa21d7f91b4-0.parquet,elephant
bucketed_images.parquet/classified_category=beaver/4f8de8f5-2abf-46a7-85df-deca84f36d5f-0.parquet,beaver
bucketed_images.parquet/classified_category=whale/487cfaf2-3a6c-4dda-8532-c64db4c120a9-0.parquet,whale


In [27]:
daft.read_parquet("bucketed_images.parquet/**").where(daft.col("classified_category") == "whale").show()

idInt64,original_categoryUtf8,imageImage[MIXED],classified_categoryUtf8
94,whale,"<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAgAAAQABAAD/wAARCACAAIADAREAAhEBAxEB/9sAQwAIBgYHBgUIBwcHCQkICgwUDQwLCwwZEhMPFB0aHx4dGhwcICQuJyAiLCMcHCg3KSwwMTQ0NB8nOT04MjwuMzQy/9sAQwEJCQkMCwwYDQ0YMiEcITIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIy/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD0nFemzlHrUjQ8UFC0AFABQAmKQBTASmAUAJimAtIBDTAQ0AMakSyJhVIRLQwHA1A0OzQUGaADNAC5oAM0AGaAGk0wDNMAzQAm6gBN1ABmgBpNBLI2NNCJM0gFBqRoN1Axd1ABmgBc0DFzQAZoAaTTAbmmITNIBc0AITQAZoAQmmhMjY1QhfNX1qQDzR60WATzR60WGHnL6igBBOPWiwEglHrRYB3mA0hgXosMY0mKAGG4Ud6pEsYblfUUWAPtK+tHKAfal9RRyiEN0vqKOUBPtK+op2AY1yvqKAMv7cT3p2JuL9tPrRYY03p9aLDGm+PrRYAW9OetFgLkN1u70WAtpNmpsUSb/elYZXmlIFAjNluSD1q0SyH7UT3qkgF+1H1p2Aabo+tFhCfaz60WEH2s+tJoLjDdn1pAV/N96oQhlzQMDLQA0S0gHq9AFmCfB60DRpQzbhUFlpWyKAGTLlTSEzGuuD1q0Sypvx3qhDDIaBjg/FAhjHJ60CFB96AELe9AECS5oGS7xQBHJJikMYsuTQBL5mBSAdHPz1oA0rafpzSYGrBJuApMomlOUqRmFfLycVZLMwNgnNUhAWpiDfigBPMoAaZcUAN8ygCsjEUhkolzQAx23d6QCA470AP3nFIBgYhqBF+2nwRTGbdpNkCoZSLkj/LSGZF3L1qkSzFuJsHgVSERJOTTuArS0ACy0XAUtnvTARmxQBDu21ACB80ARmQg0AHnEYoAlSfNMRIG3UATRHaaGM1rGXJFSykbLcx/hUjMS9OCeKpEsw53+bpTENXBoAeVyOtAEbfLQBGJGJ700BKTkUwK5YmoEJ54SmA0zBqAHBlai4DkUE9adwJx8vei4EqNmkUaVlJtYUhnQwuHj/CkMzb2Mc00SznL0KjdRQIq+eqDqKVwGLeqxIyKaYDmnHHNO4xDcqo7UXAha/AouA4yBQaQik7+Y3XFFwElmEK/eBqGwKR1Uq2BUcwyeHVvaruOxcTUt5FFwsaENyDiqA0raYAjmmBu21wNnXtUjKt5ODnmncLHLas3OQ1K4WMKSduRk1LYWIoDI8gxnrS5hWNWeIxxA5PSjmAopcksQTTTAJDnvTuBaeQmmQVpnKjgUMaM+XzJT1NQyiL7ORSsBKkJFUFy3CnTmgDSjk2Ac07gW4bjLD5qdwNy1uSE/CgaKl/cv2BqSkYFzLJIT8ppXHYqC0kmblSKlsLGlBaR2a7mIz70iWUr/Ug3yge1AjKWXkmmgB7g07gaENzuPIq7kE0jKV6ChjKoKkngVIwIX1FADZJljHGDSbCxAt0d3AqOYpRLIndhwpp3KsWrJnZxnPWncLHT2oUIMsBxVXFYlKQMDmRfzouMrPFbISdyGkBTuL+CFSFVTSA5++vzcHCnFIlma4Ock0CHxKG71SAkeAUCLwg207AMlbA60MCoxYngGkMQhsd6QIiKsx5zUNFXLMFvuIqbDua0SKi9Aa1SIuP8wRAkLRYLlaTV3XIGaRVyjLqcxPDNRcCu9/Mert+dICtJfOeNxNAEUc7Fuc0xMndi2KBEkSlec1SAleXihsCxNqKgcYpuQEC3Ikap5hlnzY1XtSAiE6MT0FAWGSSoBwRTuKwkF382MUEmpHOMDmi4hLi4Urxii4zOYhiTRYohZhnGKLARzRbxxSsBDHaYPJosFyf7OuOMUAOVQKAI5ZivQUxkLz5pAMeFh/EahgLGhHekhE+1iPvVYyB9y9zQO5GI2Y/eNIdy3bptoJLG8jvTENaQ+tAETz4p3HYjMnei47At1zjFK4WJWn4HFO4WESQnNMQ4c0AIUyKAKsqYNRcZ/9k="" alt="""" />",whale
151,whale,"<img style=""max-height:128px;width:auto"" src=""data:image/png;base64, /9j/4AAQSkZJRgABAgAAAQABAAD/wAARCACAAIADAREAAhEBAxEB/9sAQwAIBgYHBgUIBwcHCQkICgwUDQwLCwwZEhMPFB0aHx4dGhwcICQuJyAiLCMcHCg3KSwwMTQ0NB8nOT04MjwuMzQy/9sAQwEJCQkMCwwYDQ0YMiEcITIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIyMjIy/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwDgFFUQOJ20ANLZoEL1oGCg56UmIlPAqRCZwaYyUNxUsY3qaEIkHGKaAuQgnHFUBeQHHSoYxHh3UIZF5WDVoCRYc0MTF8vFIRgeVilcLkTiqTAaBTuA8HFIZOmMdKgRHJVIBqjNNgP6VIxd1FhE8Q3EUIDWtYMgVTGXQgUVJQx8UhEJTJp3ETJHRcQrJQI5thxUiIG61SGN207gOWPNFwJQMCpuMiaqAVeKGA8jNSFxAvNUBdtYyxHFCGjet0Cp+FNjG